# Comparison of topics in LDA and BERTopic models
based on the ratio of identical elements in topics

### Import packages

In [1]:
import pandas as pd
import numpy as np

### Load topic representations

In [2]:
# load file into dataframe
LDA_df = pd.read_excel("Topic_outputs/Final/LDA_25words.xlsx")

# filter to relevant columns
LDA_df = LDA_df[['0_word', '1_word','2_word','3_word', '4_word', '5_word','6_word','7_word','8_word']]

# rename columns
LDA_df = LDA_df.rename(columns={'0_word': '1: Támogatás & EU', 
                                '1_word': '2: Kereszténység & siker',
                                '2_word': '3: Migráció & krízis',
                                '3_word': '4: Támogatás & választás',
                                '4_word': '5: Szerbia & nehézség',
                                '5_word': '6: Brüsszel & külföld',
                                '6_word': '7: Hosszútávú & növekedés',
                                '7_word': '8: Energia & fejlődés',
                                '8_word': '9: Nyugat & problémák'
                               }
                      )

In [3]:
#load file into dataframe
BERT_df = pd.read_excel("Topic_outputs/Final/BERT_LDA_25words.xlsx")

#filter to relevant columns
BERT_df = BERT_df[['-1_word','0_word', '1_word','2_word','3_word', '4_word', '5_word','6_word','7_word','8_word']]

# rename columns
BERT_df = BERT_df.rename(columns={'-1_word': 'Outlier topik',
                                  '0_word': '1: Sikeres fejlődés',
                                  '1_word': '2: Európa',
                                  '2_word': '3: Orosz-ukrán háború',
                                  '3_word': '4: Hazafiság',
                                  '4_word': '5: Covid',
                                  '5_word': '6: Egyház',
                                  '6_word': '7: Sport',
                                  '7_word': '8: Kína',
                                  '8_word': '9: Ipar'
                                 }
                        )

## Calculate topic similarity

In [4]:
# Check for duplicates in topics
for topic in BERT_df.columns:
    if len(set(BERT_df[topic].to_list())) != len(BERT_df[topic].to_list()):
        print(f'There are duplicate values in {topic} topic.')

In [5]:
# Check for duplicates in topics
for topic in LDA_df.columns:
    if len(set(LDA_df[topic].to_list())) != len(LDA_df[topic].to_list()):
        print(f'There are duplicate values in {topic} topic.')

### Function

In [6]:
# Function to create a matrix of the ratio of identical elements in topics
def similarity_matrix(df_A, df_B):
    sm = []
    
    for column_A in df_A.columns:
        sm_topic = []
        for column_B in df_B.columns:
            all_words = df_A[column_A].to_list() + df_B[column_B].to_list()
            unique_words = set(all_words)
            similarity = 1-(len(unique_words)/len(all_words))
            sm_topic.append(similarity)
        sm.append(sm_topic)
    
    return sm

### Based on LDA

In [7]:
sim_matrix = similarity_matrix(LDA_df,BERT_df)

lda_bert_compare = []

for topic_num in range(0,9,1):

    most_similar_topic = np.argmax(sim_matrix[topic_num])
    
    lda_bert_compare.append(
        {'LDA topic': LDA_df.columns[topic_num],
         'Most similar BERT_LDA topic': BERT_df.columns[most_similar_topic],
         'Similarity ratio': round(sim_matrix[topic_num][most_similar_topic],3)
        }
    )

pd.DataFrame(lda_bert_compare)

,LDA topic,Most similar BERT_LDA topic,Similarity ratio
0,1: Támogatás & EU,2: Európa,0.16
1,2: Kereszténység & siker,Outlier topik,0.22
2,3: Migráció & krízis,Outlier topik,0.18
3,4: Támogatás & választás,Outlier topik,0.22
4,5: Szerbia & nehézség,Outlier topik,0.22
5,6: Brüsszel & külföld,Outlier topik,0.22
6,7: Hosszútávú & növekedés,Outlier topik,0.26
7,8: Energia & fejlődés,1: Sikeres fejlődés,0.20
8,9: Nyugat & problémák,Outlier topik,0.20


### Based on BERT

In [8]:
sim_matrix = similarity_matrix(BERT_df, LDA_df)

bert_lda_compare = []

for topic_num in range(0,10,1):

    most_similar_topic = np.argmax(sim_matrix[topic_num])
    
    bert_lda_compare.append(
        {'BERT_LDA topic': BERT_df.columns[topic_num],
         'Most similar LDA topic': LDA_df.columns[most_similar_topic],
         'Similarity ratio': round(sim_matrix[topic_num][most_similar_topic],2)
        }
    )

pd.DataFrame(bert_lda_compare)

,BERT_LDA topic,Most similar LDA topic,Similarity ratio
0,Outlier topik,7: Hosszútávú & növekedés,0.26
1,1: Sikeres fejlődés,7: Hosszútávú & növekedés,0.24
2,2: Európa,1: Támogatás & EU,0.16
3,3: Orosz-ukrán háború,9: Nyugat & problémák,0.10
4,4: Hazafiság,3: Migráció & krízis,0.14
5,5: Covid,3: Migráció & krízis,0.04
6,6: Egyház,4: Támogatás & választás,0.08
7,7: Sport,6: Brüsszel & külföld,0.04
8,8: Kína,5: Szerbia & nehézség,0.10
9,9: Ipar,1: Támogatás & EU,0.04
